# **MNIST Dataseti ile Deep Learning Uygulaması**

In [20]:
from keras.datasets import mnist

In [21]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

print("ilk datanin shape'i:", x_train[0].shape) # (28, 28)
print("ilk datanin icerigi:", y_train[0]) # 5

ilk datanin shape'i: (28, 28)
ilk datanin icerigi: 5


Normalization

In [22]:
x_train = x_train.reshape((60000, 28*28)) #Goruntuyu vektore donusturur
x_train = x_train.astype('float32') / 255 #Normalize eder
x_test = x_test.reshape((10000, 28*28)) 
x_test = x_test.astype('float32') / 255

print("Normalizasyon sonrasi shape:", x_train.shape) # (60000, 784)

Normalizasyon sonrasi shape: (60000, 784)


Devamında ağın yapısını tanımlamamız gerekiyor.

In [23]:
from keras import models
from keras import layers

`from keras.models import Sequential` gibi de kullanabiliriz.

Modeli kullanmadan önce `models.Sequential()` gibi tanımlamalıyız.

In [24]:
model = models.Sequential()

In [25]:
#Giris katmani
model.add(layers.Dense(16, 
                       activation='relu', 
                       input_shape=(28*28,)))

In [26]:
model.add(layers.Dense(16,
                       activation='relu'))

Çıkış katmanımızda 10 adet çıkış olmalı.  
0,1,2,3,4,5,6,7,8,9

Bu, **multi class classification** olarak geçer. Dolayısıyla çıkış katmanımızda activation function olarak **softmax** tercih ediyoruz.

**Binary Classification**'da ise genelde **sigmoid** tercih edilir.

In [27]:
#Cikis katmani
model.add(layers.Dense(10, #cikis sayisina esit olmali
                       activation="softmax"))

`categorical_crossentropy` kullanabilmemiz için, çıkış datalarımız olan *y_train* ve *y_test* datalarını categorical'a dönüştürmemiz gerekiyor.

Örneğin, **5** olarak değil **[0,0,0,0,0,1,0,0,0,0]** olarak modele vermemiz gerekiyor.

`sparse_categorical_crossentropy` bu işlemi kendisi yapar. `to_categorical` yapmamıza gerek kalmaz.

In [28]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [29]:
y_train.shape

(60000, 10)

In [30]:
y_train[0,:] # 5 vardi

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

Optimizer olarak *Stochastic Gradient Descent (SGD)* kullanıyoruz.

Loss function olarak *Categorical Cross Entropy* kullanıyoruz.  
Problem tipi Regresyon ise genelde *mse, mae* gibi regresyon fonksiyonları kullanılır.  
Multi class classification'da ise *cross entropy* veya *sparse categorical cross entropy* kullanılır.

Metrics ise ekrana yazdıracağımız değeri belirtir.




In [31]:
model.compile(optimizer="sgd", 
              loss="categorical_crossentropy",
              metrics=["accuracy"])

Modelin yapısını özet olarak görelim;

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 16)                12560     
_________________________________________________________________
dense_10 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_11 (Dense)             (None, 10)                170       
Total params: 13,002
Trainable params: 13,002
Non-trainable params: 0
_________________________________________________________________


Birinci katmanın çıkışı 16 olarak gözüküyor.  

Neden 12560 parametre var?  
Girişimiz 784 elemanlı.  
Her bir hücreden 784x16 adet *weights* bağlantısı var demektir.  
*Bias*'lar da var; 16 adet. (her bir hücre için 1 adet.)  
784x16+16 = 12560 parametreye denk gelir.

---

Sıradaki adımımız modeli eğitmek.

In [32]:
model.fit(x_train, y_train, epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 1s 1ms/step - loss: 1.3326 - accuracy: 0.5856
Epoch 2/5
938/938 [==============================] - 1s 1ms/step - loss: 0.5173 - accuracy: 0.8566
Epoch 3/5
938/938 [==============================] - 1s 1ms/step - loss: 0.4049 - accuracy: 0.8848
Epoch 4/5
938/938 [==============================] - 2s 2ms/step - loss: 0.3555 - accuracy: 0.8983
Epoch 5/5
938/938 [==============================] - 1s 1ms/step - loss: 0.3252 - accuracy: 0.9072


Eğitilmiş ağın performansını ölçelim.

Eğitim bittikten sonra eğitimde kullanılmamış görüntülerle ağın başarımı belli olur.

In [33]:
test_loss, test_acc = model.evaluate(x_test, y_test) #test_images, test_labels

313/313 [==============================] - 0s 929us/step - loss: 0.2978 - accuracy: 0.9124
